<a href="https://colab.research.google.com/github/surayajohari/STQD6324-Data-Management/blob/main/suraya_assignment4_stqd6324.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NAME**: SURAYA ADNIN BINTI JOHARI

**NO. MATRIKS**: P137383

**SUBJECT CODE**: STQD6324

**SUBJECT NAME**: DATA MANAGEMENT

**ASSIGNMENT 4**: Cassandra, HBase, MongoDB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. To import Relevant libraries

In [89]:
# Import relevant libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count
from cassandra.cluster import Cluster

# 2. Initialize Spark Session & Cassandra cluster

In [92]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("MovieLensAnalysis") \
    .config("spark.cassandra.connection.host", "127.0.0.1:9042") \
    .getOrCreate()

# Cassandra keyspace and table names
KEYSPACE = "movielens"
USER_TABLE = "users"
MOVIE_TABLE = "movies"
RATING_TABLE = "ratings"

# Initialize Cassandra Cluster
cluster = Cluster(['127.0.0.1:9042'])
session = cluster.connect()

UnresolvableContactPoints: {}

In [93]:
# Parse the local u.user file and upload it to HDFS
def parse_and_upload_to_hdfs(local_file_path, hdfs_path):
    import shutil
    shutil.copy(local_file_path, hdfs_path)
    print(f"File {local_file_path} uploaded to HDFS path {hdfs_path}")

# Load RDD from HDFS
def load_rdd_from_hdfs(file_path):
    return spark.sparkContext.textFile(file_path)

# Convert RDD to Dataframe
def rdd_to_dataframe(rdd, schema):
    return spark.createDataFrame(rdd.map(lambda x: x.split('|')).map(lambda x: (x[0], x[1], x[2], x[3])), schema)

# Write DataFrame to Cassandra.
def write_dataframe_to_cassandra(df, table_name):
    df.write.format("org.apache.spark.sql.cassandra").options(table=table_name, keyspace=KEYSPACE).mode("append").save()

# Read DataFrame from Cassandra table
def read_from_cassandra(table_name):
    return spark.read.format("org.apache.spark.sql.cassandra").options(table=table_name, keyspace=KEYSPACE).load()

# Calculate average rating for each movie
def calculate_avg_rating_per_movie():
    ratings_df = read_from_cassandra(RATING_TABLE)

    avg_ratings_df = ratings_df.groupBy("movieId").agg(avg("rating").alias("avg_rating"))
    avg_ratings_df.show(10)

# Identify top ten movies with the highest average ratings.
def top_ten_movies_by_avg_rating():
    ratings_df = read_from_cassandra(RATING_TABLE)

    avg_ratings_df = ratings_df.groupBy("movieId").agg(avg("rating").alias("avg_rating"))
    top_ten_movies_df = avg_ratings_df.orderBy(col("avg_rating").desc()).limit(10)
    top_ten_movies_df.show()

# Find users who have rated at least 50 movies and identify their favourite movie genres.
def find_users_with_50_ratings():
    ratings_df = read_from_cassandra(RATING_TABLE)
    users_df = read_from_cassandra(USER_TABLE)

    user_ratings_count_df = ratings_df.groupBy("userId").agg(count("movieId").alias("rating_count"))
    active_users_df = user_ratings_count_df.filter(col("rating_count") >= 50)

    # Join with user data to find favourite genres (assuming genres are in USER_TABLE)
    user_genres_df = active_users_df.join(users_df, "userId")
    user_genres_df.show()

# Find all users with age less than 20 years old.
def find_users_under_20():
    users_df = read_from_cassandra(USER_TABLE)
    young_users_df = users_df.filter(col("age") < 20)
    young_users_df.show()

# Find all users with occupation "scientist" and age between 30 and 40 years old.
def find_scientists_between_30_40():
    users_df = read_from_cassandra(USER_TABLE)
    scientists_df = users_df.filter((col("occupation") == "scientist") & (col("age") >= 30) & (col("age") <= 40))
    scientists_df.show()

if __name__ == "__main__":
    # Define file paths
    local_file_path = "/content/drive/MyDrive/STQD6324/u.user"
    hdfs_path = "hdfs:///user/hadoop/u.user"

    # Parse the file and upload to HDFS
    parse_and_upload_to_hdfs(local_file_path, hdfs_path)

    # Define schemas
    user_schema = ["userId", "age", "gender", "occupation"]
    rating_schema = ["userId", "movieId", "rating", "timestamp"]

    # Load RDDs from HDFS
    user_rdd = load_rdd_from_hdfs(hdfs_path)
    rating_rdd = load_rdd_from_hdfs("hdfs:///user/hadoop/ratings.dat")  # Adjust path as necessary

    # Convert RDDs to DataFrames
    user_df = rdd_to_dataframe(user_rdd, user_schema)
    rating_df = rdd_to_dataframe(rating_rdd, rating_schema)

    # Write DataFrames to Cassandra
    write_dataframe_to_cassandra(user_df, USER_TABLE)
    write_dataframe_to_cassandra(rating_df, RATING_TABLE)

    # Execute queries
    calculate_avg_rating_per_movie()
    top_ten_movies_by_avg_rating()
    find_users_with_50_ratings()
    find_users_under_20()
    find_scientists_between_30_40()

    # Stop the SparkSession
    spark.stop()


FileNotFoundError: [Errno 2] No such file or directory: 'hdfs:///user/hadoop/u.user'

In [91]:
sudo systemctl status cassandra

SyntaxError: invalid syntax (<ipython-input-91-75311a629987>, line 1)